<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/End_to_End_Python_Script_with_Notebook_Compatibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import sys

def build_parser():
    parser = argparse.ArgumentParser(prog="colab_kernel_launcher.py")
    subparsers = parser.add_subparsers(dest="cmd", help="Available subcommands")

    # 'single' subcommand
    p_single = subparsers.add_parser("single", help="Run single metric evaluation")
    p_single.add_argument("--mass", type=float, required=True, help="Mass (kg)")
    p_single.add_argument("--energy-density", type=float, required=True, help="Energy density (arbitrary units)")
    p_single.add_argument("--distance", type=float, required=True, help="Distance (m)")
    p_single.set_defaults(func=handle_single)

    # 'sweep' subcommand
    p_sweep = subparsers.add_parser("sweep", help="Sweep over energy density range")
    p_sweep.add_argument("--mass", type=float, required=True)
    p_sweep.add_argument("--distance", type=float, required=True)
    p_sweep.add_argument("--energy-density-range", type=float, nargs=3, metavar=("START", "END", "NUM"))
    p_sweep.set_defaults(func=handle_sweep)

    return parser


def sanitize_argv(argv):
    """Remove notebook-injected args like '-f' and JSON kernel paths."""
    return [
        arg for i, arg in enumerate(argv)
        if not (
            arg == "-f" or
            (i > 0 and argv[i-1] == "-f") or
            arg.endswith(".json") or
            "/jupyter/runtime/kernel" in arg
        )
    ]


def handle_single(args):
    print(f"[single] Mass={args.mass}, EnergyDensity={args.energy_density}, Distance={args.distance}")


def handle_sweep(args):
    start, end, num = args.energy_density_range
    print(f"[sweep] Mass={args.mass}, Distance={args.distance}")
    print(f"EnergyDensity Range: {start} to {end}, {int(num)} steps")


def main(argv=None):
    if argv is None:
        argv = sanitize_argv(sys.argv[1:])
    else:
        argv = sanitize_argv(argv)

    parser = build_parser()
    try:
        args = parser.parse_args(argv)

        if not hasattr(args, "func"):
            parser.print_help()
        else:
            args.func(args)
    except Exception as e:
        print(f"Error: {e}", file=sys.stderr)


if __name__ == "__main__":
    main()